# chapter4
これまではスカラを扱ってきたが機械学習で中心となるのはテンソル(ベクトルや行列)なのでDeZeroでテンソルを扱えるように拡張していく\
→基本的にはNumPyの機能をVariableでも使えるように実装していく流れ

## Step37
### 37.1
これまではスカラを想定\
sin関数を例に用いて、テンソルが入力された場合Variableがどんな計算になるのかをみる

```python
class Sin(Function):
    def forward(self, x):
        y = np.sin(x)
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = gy * np.cos(x)
        return gx

def sin(x):
    return Sin()(x)

In [1]:
import numpy as np
import dezero.functions as F
from dezero import Variable

x = Variable(np.array(1.0))
y = F.sin(x)
print(y)

/Volumes/yason01_ssd/data/05_輪読会/zerotsuku3/chapter4/step37~39/dezero/datasets.py:234: SyntaxWarning: "is" with a literal. Did you mean "=="?
  return fine_labels if label_type is 'fine' else coarse_labels


variable(0.8414709848078965)


入力がテンソルの場合

In [106]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sin(x)
print(y)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])


xの各要素に対してsin関数が適用されている\
→出力はxと同じshape

続いてテンソル同士の足し算をみる

In [107]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
c = Variable(np.array([[10, 20, 30], [40, 50, 60]]))
y = x + c
print(y)

variable([[11 22 33]
          [44 55 66]])


要素の同じ場所同士の計算になっている\
→出力はxとcと同じshape

In [108]:
d = Variable(np.array([[10, 20], [30, 40]]))
# z = x + d
# print(z) #error

NumPyにshapeが違うもの同士の計算を助けるブロードキャストという機能があるが解説はStep40

### 37.2

ここではテンソルでのバックプロパゲーションも正しく動作することを確認する\
\
下記、Step39のsum関数を使う→Step39で解説\
これはテンソルの全ての要素の和をとり、1つのスカラを出力する

dezero/functions.py

```python
class Sum(Function):
    def __init__(self, axis, keepdims):
        self.axis = axis
        self.keepdims = keepdims
    
    def forward(self, x):
        self.x_shape = x.shape
        y = x.sum(axis=self.axis, keepdims=self.keepdims)
        return y
    
    def backward(self, gy):
        gy = utils.reshape_sum_backward(gy, self.x_shape, self.axis, self.keepdims)
        gx = broadcast_to(gy, self.x_shape)
        return gx

def sum(x, axis=None, keepdims=False):
    return Sum(axis, keepdims)(x)

In [109]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
c = Variable(np.array([[10, 20, 30], [40, 50, 60]]))
t = x + c
y = F.sum(t)
print(t)
print(y)

variable([[11 22 33]
          [44 55 66]])
variable(231)


In [110]:
y.backward(retain_grad=True)
print(y.grad)
print(t.grad)
print(x.grad)
print(c.grad)

variable(1)
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])


In [111]:
print(y)
print(t)
print(x)
print(c)

variable(231)
variable([[11 22 33]
          [44 55 66]])
variable([[1 2 3]
          [4 5 6]])
variable([[10 20 30]
          [40 50 60]])


注目ポイントは\
要素ごとの逆伝播になっていること→この後の補足で見ていく\
微分前と後でshapeが同じになっていること→Step38で見ていく

### 37.3 【補足】テンソルを使用したときのバックプロパゲーション  

ここでの問題は「テンソルのバックプロパゲーションは要素ごとの微分」ということを証明していく\
\
まず前提の確認としてx, yを次のようにおくと

### $$y = (y_1 \quad y_2 \quad y_3 \quad ... \quad y_n)$$
### $$x = (x_1 \quad x_2 \quad x_3 \quad ... \quad x_n)$$

### $$\frac{∂y}{∂x}=\begin{pmatrix}\frac{∂y_1}{∂x_1}&\frac{∂y_1}{∂x_2}&...&\frac{∂y_1}{∂x_n}\\\frac{∂y_2}{∂x_1}&\frac{∂y_2}{∂x_2}&...&\frac{∂y_2}{∂x_n}\\...&...&...&...\\\frac{∂y_n}{∂x_1}&\frac{∂y_n}{∂x_2}&...&\frac{∂y_n}{∂x_n}\\\end{pmatrix}$$

yをスカラとすると1×nのヤコビ行列(ベクトル)と考えることができる

### $$\frac{∂y}{∂x} = (\frac{∂y}{∂x_1} \quad \frac{∂y}{∂x_2} \quad ... \quad \frac{∂y}{∂x_n})$$

では次の合成関数について考える

### $$y = F(x)$$

### $$x → A() → a → B() → b → C() → y$$

チェインルールにより次の式で表すことができる

### $$\frac{∂y}{∂x} = \frac{∂y}{∂b}\frac{∂b}{∂a}\frac{∂a}{∂x}$$

上記の式の右辺のそれぞれは、前提確認した内容よりヤコビ行列の積である\
ここで行列の積は結合法則(どっちから先に計算しても答えが同じ)が成り立つので

### $$①\qquad\frac{∂y}{∂x} = \frac{∂y}{∂b}(\frac{∂b}{∂a}\frac{∂a}{∂x})$$
### $$②\qquad\frac{∂y}{∂x} = (\frac{∂y}{∂b}\frac{∂b}{∂a})\frac{∂a}{∂x}$$

上記の2パターンの選択肢があることになる\
行列のshapeに注目してみるとyはスカラ、a,b,c,xは要素数nのベクトルなので

### $$(1×n) = (1×n)(n×n)(n×n)$$

行列の積のshapeには下記のような特性がある
### $$(2×3) × (3×4) = (2×4)$$

結局①と②の違いは楽に計算できるかどうか→②の方が計算効率が良い
### $$①\qquad(n×n) × (n×n) → (1×n) × (n×n) → (1×n)$$
### $$②\qquad(1×n) × (n×n) → (1×n) × (n×n) → (1×n)$$

$$$$

$$$$

結果がわかれば、わざわざ行列の積の計算をしなくてもOK\
要素で先に計算を進められるところを探すと\
x1はa1、x2はa2、…、xnはanにしか影響を与えないため次のような対角行列になる

### $$\frac{∂a}{∂x}=\begin{pmatrix}\frac{∂a_1}{∂x_1}&0&...&0\\0&\frac{∂a_2}{∂x_2}&...&0\\...&...&...&...\\0&0&...&\frac{∂y_n}{∂x_n}\\\end{pmatrix}$$

### $$\frac{∂y}{∂a}\frac{∂a}{∂x}=(\frac{∂y}{∂a_1} \quad \frac{∂y}{∂a_2} \quad ... \quad \frac{∂y}{∂a_n})  \begin{pmatrix}\frac{∂a_1}{∂x_1} & 0 & ... & 0\\ 0 & \frac{∂a_2}{∂x_2} & ... & 0\\ ... & ... & ... & ...\\ 0 & 0 & ... & \frac{∂y_n}{∂x_n}\\ \end{pmatrix}$$

### $$=( \frac{∂y}{∂a_1} \frac{∂a_1}{∂x_1} \quad \frac{∂y}{∂a_2} \frac{∂a_2}{∂x_2} \quad ... \quad \frac{∂y}{∂a_n} \frac{∂a_n}{∂x_n})$$

つまり要素ごとに微分を求めて、その微分を要素ごとにかけることで求められる

$$$$

$$$$

$$$$

## Step38

テンソルでの実行について、要素ごとに計算を行うものは使えることがわかった\
なので要素ごとに計算を行わない関数について実装をしていく

### 38.1 reshapeの実装

In [112]:
import numpy as np

x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.reshape(x, (6,))
print(y)

[1 2 3 4 5 6]


In [113]:
print(x.shape)
print(y.shape)

(2, 3)
(6,)


NumPyは形を変えるだけだが、DeZeroでは逆伝播も考慮する必要がある

dezero/fuctions.py

```python
class Reshape(Function):
    def __init__(self, shape):
        self.shape = shape
    
    def forward(self, x):
        self.x_shape = x.shape
        y = x.reshape(self.shape)
        return y
    
    def backward(self, gy):
        return reshape(gy, self.x_shape)

def reshape(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return Reshape(shape)(x)

In [114]:
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.reshape(x, (6,))
y.backward(retain_grad=True)
print(x)
print(x.grad)
print(y)
print(y.grad)

variable([[1 2 3]
          [4 5 6]])
variable([[1 1 1]
          [1 1 1]])
variable([1 2 3 4 5 6])
variable([1 1 1 1 1 1])


### 38.2 reshapeをより使いやすく
NumPyのreshapeでは下記のような使い方ができる

In [115]:
x = np.random.rand(1, 2, 3)
print(x)
print()

y = x.reshape((2, 3)) #タプル
print(y)
print()

y = x.reshape([2, 3]) #リスト
print(y)
print()

y= x.reshape(2, 3) #そのまま
print(y)

[[[0.13786097 0.48894349 0.48505895]
  [0.83786424 0.44833111 0.18961326]]]

[[0.13786097 0.48894349 0.48505895]
 [0.83786424 0.44833111 0.18961326]]

[[0.13786097 0.48894349 0.48505895]
 [0.83786424 0.44833111 0.18961326]]

[[0.13786097 0.48894349 0.48505895]
 [0.83786424 0.44833111 0.18961326]]


NumPyのように柔軟な引数を設定し、さらにVariableから直接reshapeを実行できるようにする

dezero/core.py

```python
class Variable:
    ...

    def reshape(self, *shape):
        if len(shape)==1 and isinstance(shape[0], (tuple, list)):
            shape = shape[0]
        return dezero.fuctions.reshape(self, shape)

In [116]:
x = Variable(np.random.randn(1, 2, 3))
print(x)
print()

y = x.reshape((2, 3))
print(y)
print()

y = x.reshape([2, 3])
print(y)
print()

y = x.reshape(2, 3)
print(y)

variable([[[ 1.32640131 -0.29508255  1.09494033]
           [ 0.08091145  1.42464873  0.27643882]]])

variable([[ 1.32640131 -0.29508255  1.09494033]
          [ 0.08091145  1.42464873  0.27643882]])

variable([[ 1.32640131 -0.29508255  1.09494033]
          [ 0.08091145  1.42464873  0.27643882]])

variable([[ 1.32640131 -0.29508255  1.09494033]
          [ 0.08091145  1.42464873  0.27643882]])


### 38.3 行列の転置
NumPyでは下記のように実装できる

In [117]:
x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.transpose(x)
print(x)
print(x.shape)
print()
print(y)
print(y.shape)

[[1 2 3]
 [4 5 6]]
(2, 3)

[[1 4]
 [2 5]
 [3 6]]
(3, 2)


dezero/functions.py

```python
class Transpose(Function):
    def forward(self, x):
        y = np.transpose(x)
        return y
    
    def backward(self, gy):
        gx = transpose(gy)
        return gx

def transpose(x):
    return Transpose()(x)

In [118]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.transpose(x)
y.backward(retain_grad=True)
print(x)
print(x.grad)
print(y)
print(y.grad)

variable([[1 2 3]
          [4 5 6]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 4]
          [2 5]
          [3 6]])
variable([[1 1]
          [1 1]
          [1 1]])


dezero/core.py

```python
class Variable:
    ...

    def transpose(self):
        return dezero.functions.transpose(self)
    
    @property
    def T(self):
        return dezero.functions.transpose(self)

In [119]:
x = Variable(np.random.rand(2, 3))
print(x)
print()
y = x.transpose()
print(y)
print()

y = x.T
print(y)

variable([[0.90403417 0.27543661 0.0813162 ]
          [0.05733018 0.76421896 0.26333158]])

variable([[0.90403417 0.05733018]
          [0.27543661 0.76421896]
          [0.0813162  0.26333158]])

variable([[0.90403417 0.05733018]
          [0.27543661 0.76421896]
          [0.0813162  0.26333158]])


### 38.4 補足
NumPyのtransposeにはデータの順番をインデックス番号の指定でも入れ替えることができる

In [120]:
x = np.random.rand(1, 2, 3, 4)
y = x.transpose(1, 0, 3, 2)

In [121]:
print(x.shape)
print(x)

(1, 2, 3, 4)
[[[[0.68351663 0.99728663 0.99334428 0.34694907]
   [0.61680196 0.42726231 0.57509879 0.34815371]
   [0.8604593  0.37267718 0.46107891 0.70091219]]

  [[0.30203624 0.62865941 0.23951689 0.02147002]
   [0.56705124 0.24271007 0.25969397 0.73536797]
   [0.9968003  0.6008376  0.34978126 0.79782717]]]]


In [122]:
print(y.shape)
print(y)

(2, 1, 4, 3)
[[[[0.68351663 0.61680196 0.8604593 ]
   [0.99728663 0.42726231 0.37267718]
   [0.99334428 0.57509879 0.46107891]
   [0.34694907 0.34815371 0.70091219]]]


 [[[0.30203624 0.56705124 0.9968003 ]
   [0.62865941 0.24271007 0.6008376 ]
   [0.23951689 0.25969397 0.34978126]
   [0.02147002 0.73536797 0.79782717]]]]


dezero/functions.py

```python
class Transpose(Function):
    def __init__(self, axes=None):
        self.axes = axes

    def forward(self, x):
        y = x.transpose(self.axes)
        return y

    def backward(self, gy):
        if self.axes is None:
            return transpose(gy)

        axes_len = len(self.axes)
        inv_axes = tuple(np.argsort([ax % axes_len for ax in self.axes]))
        return transpose(gy, inv_axes)

$$$$

$$$$

$$$$

## Step39

Step37でちらっとでてきたsum関数を実装する

### 39.1

dezero/core_simple.py

```python
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

ベクトルの足し算でも使えるようにsum関数を実装する

### 39.2

指定された形状に要素をコピーするbroadcast_toをStep40から先取り

dezero/functions.py

```python
class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape
    
    def forward(self, x):
        self.x_shape = x.shape
        y = np.broadcast_to(x, self.shape)
        return y
    
    def backward(self, gy):
        gx = sum_to(gy, self.x_shape)
        return gx

def broadcast_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return BroadcastTo(shape)(x)

dezero/functions.py

```python
class Sum(Function):
    def forward(self, x):
        self.x_shape = x.shape
        y = x.sum()
        return y
    
    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx

def sum(x):
    return Sum()(x)

In [123]:
import numpy as np
from dezero import Variable
import dezero.functions as F

# 入力が1次元の場合
x = Variable(np.array([1, 2, 3, 4, 5, 6]))
y = F.sum(x)
y.backward(retain_grad=True)
print(x)
print(x.grad)
print()
print(y)
print(y.grad)

variable([1 2 3 4 5 6])
variable([1 1 1 1 1 1])

variable(21)
variable(1)


In [124]:
# 入力が2次元の場合
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sum(x)
y.backward(retain_grad=True)
print(x)
print(x.grad)
print()
print(y)
print(y.grad)

variable([[1 2 3]
          [4 5 6]])
variable([[1 1 1]
          [1 1 1]])

variable(21)
variable(1)


### 39.3 さらに機能を拡張

NumPyのsum機能は和を求める際に軸を指定できるなど、高機能である

#### axis
足し算の軸の方向を指定

In [125]:
import numpy as np

# 行方向の足し算
x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.sum(x, axis=0) #デフォルトはNone
print(y)
print(x.shape, ' -> ', y.shape)

[5 7 9]
(2, 3)  ->  (3,)


In [126]:
# 列方向の足し算
x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.sum(x, axis=1)
print(y)
print(x.shape, ' -> ', y.shape)

[ 6 15]
(2, 3)  ->  (2,)


#### keepdims
入力と出力を同じ次元に保つかどうか

In [127]:
import numpy as np
x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.sum(x, keepdims=True)
print(y)
print(y.shape)

[[21]]
(1, 1)


In [128]:
x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.sum(x, keepdims=False)
print(y)
print(y.shape)

21
()


dezero/functions.py

```python
class Sum(Function):
    def __init__(self, axis, keepdims):
        self.axis = axis
        self.keepdims = keepdims
    
    def forward(self, x):
        self.x_shape = x.shape
        y = x.sum(axis=self.axis, keepdims=self.keepdims)
        return y
    
    def backward(self, gy):
        gy = utils.reshape_sum_backward(gy, self.x_shape, self.axis, self.keepdims)
        gx = broadcast_to(gy, self.x_shape)
        return gx

def sum(x, axis=None, keepdims=False):
    return Sum(axis, keepdims)(x)

broadcast_toについてはStep40で実装予定\
utils.reshape_sum_backwardについてはサソリマークで注意書きされているがNumPy関連の問題への対応なので説明は省略

Variableのメソッドとして使用できるようにcore.pyにも追記

dezero/core.py

```python
class Variable:
    ...
    def sum(self, axis=None, keepdims=False):
        return dezero.functions.sum(self, axis, keepdims)

In [129]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sum(x, axis=0)
y.backward(retain_grad=True)
print(y)
print(y.grad)
print()
print(x)
print(x.grad)

variable([5 7 9])
variable([1 1 1])

variable([[1 2 3]
          [4 5 6]])
variable([[1 1 1]
          [1 1 1]])


In [130]:
x = Variable(np.random.randn(2, 3, 4, 5))
y = x.sum(keepdims=True)
print(x)
print()
print(y)
print(y.shape)

variable([[[[ 2.83872847e-01  1.14374700e+00 -6.98194110e-01  2.54586628e-01
             -2.46439872e-01]
            [-7.34161161e-01 -9.21126990e-01 -1.22118448e+00  3.74027512e-01
             -5.75123476e-01]
            [ 7.20868891e-02  1.91964095e-01  1.27598162e+00 -6.17067937e-02
             -6.85872141e-01]
            [-3.46311885e-01 -1.81035918e+00  6.55874815e-02 -1.43031028e+00
             -9.17710277e-01]]
         
           [[ 6.73235967e-01  4.32489765e-01  3.55907399e-02 -9.47901839e-01
              8.11712306e-01]
            [-1.61597382e+00  2.06240647e+00  2.55862344e+00  1.71651357e+00
             -2.01825926e-01]
            [-1.97834353e+00 -8.82116269e-01  8.56137493e-02  1.07260402e+00
              4.96702305e-01]
            [ 6.07711948e-01 -9.20848480e-01  4.47766403e-01  2.19526914e-01
             -5.44340288e-01]]
         
           [[ 1.34895361e+00 -1.18377267e+00 -9.07738645e-01  7.00663318e-01
             -4.49025052e-01]
            [ 1